In [ ]:


import pyarrow.csv as pv
import pyarrow.parquet as pq
import os
import pandas as pd
# 定义输入和输出文件夹
input_folder = "data2"  # CSV.gz 文件所在目录
output_folder = "data"  # Parquet 存放目录

# 确保输出目录存在
# os.makedirs(output_folder, exist_ok=True)

# # # 遍历 data2 目录下所有 .csv.gz 文件
# for file_name in os.listdir(input_folder):
#     if file_name.endswith(".csv.gz"):  # 仅处理 CSV.gz 文件
#         input_path = os.path.join(input_folder, file_name)
#         output_path = os.path.join(output_folder, file_name.replace(".csv.gz", ".parquet"))

#         print(f"正在转换: {input_path} -> {output_path}")
        
#         # 读取 CSV.gz 为 PyArrow 表（不会一次性加载到内存）
#         table = pv.read_csv(input_path)
        
#         # 保存为 Parquet
#         pq.write_table(table, output_path)

# print("所有文件已转换完成")




def _parse_slice(s):
    """Convert int/slice string to proper slice object"""
    if isinstance(s, int):
        return slice(s, s+1)  # Single item slice
    elif isinstance(s, str):
        if ':' in s:
            parts = [int(x) if x else None for x in s.split(':')]
            return slice(*parts)
        else:
            return slice(int(s), int(s)+1)
    return s

def investigate(table, row=None, col=None):
    """
    Analyze parquet file with flexible slicing:
    - row/col can be int (single item) or slice spec (e.g., '2:5')
    - Returns: (submatrix, metadata) where metadata is (colnames, nrow, ncol)
    
    Example usage:
    investigate("data.parquet", row=1, col=1)  # 2nd row & 2nd column
    investigate("data.parquet", row="1:4")     # rows 2-4 (Python slice)
    """
    # Read entire table
    nrow, ncol = table.num_rows, table.num_columns
    
    # Parse slicing parameters
    row_slice = _parse_slice(row) if row is not None else slice(None)
    col_slice = _parse_slice(col) if col is not None else slice(None)
    
    # Handle column selection by index/name
    if isinstance(col_slice, int):
        col_idx = col_slice
        if col_idx >= ncol:
            raise ValueError(f"Column index {col_idx} out of range ({ncol} cols)")
        col_names = [table.column_names[col_idx]]
    else:
        col_names = table.column_names[col_slice]
    
    # Slice the table
    sub_table = table.slice(row_slice.start, row_slice.stop).select(col_names)
    
    # Convert to pandas for display
    df = sub_table.to_pandas()
    
    # Prepare metadata
    metadata = (table.column_names, nrow, ncol)
    
    return df, metadata








In [ ]:
#Read data

lab = pq.read_table('data/lab.parquet')
apachePatientResult=pq.read_table('data/apachePatientResult.parquet')

apacheApsVar = pq.read_table('data/apacheApsVar.parquet')

patient = pq.read_table('data/patient.parquet')
# nurseAssessment = pq.read_table('data/nurseAssessment.parquet')

In [ ]:
import gc
gc.collect()  # 手动清理内存

patient  unitdischargeoffset ->Y
一个patient进入icu次数  unitvisitnumber

sample_n
patient: subclass desease apacheadmissiondx挑选病理种类得到subsample

hospital id, give subsample*****
X selection:
patient age, gender, ethnicity人种,admissionweight,dischargeheight,

vitalperidic, 第一次测量的各种值

lab
apachpatientresult: apachescore,select apachversion IVa, predictedicumortality,predictediculos

一个病人会有很多病，选择第一次进ICU的病,在patient中有.



In [ ]:
#读取关键数据
dfpatient,metapatient=investigate(patient,row="0:3000000")
dflab,metalab=investigate(lab,row="0:40000000")
dfAPR,metaAPR=investigate(apachePatientResult,row="0:3000000")
most_frequent_id = dfpatient["hospitalid"].value_counts().idxmax()

# 提取同一 hospitalid 的数据


In [58]:
dfpatient['hospitalid'].unique()

array([ 59,  60,  73,  67,  66,  63,  71,  56,  68,  69,  58,  61,  92,
        79, 108,  83,  95,  84,  93,  86,  90,  85, 102,  96,  94,  91,
       110, 122, 123, 125, 135, 112, 133, 120, 131, 138, 115, 136, 144,
       143, 142, 140, 141, 175, 165, 152, 167, 171, 154, 148, 157, 176,
       164, 155, 146, 174, 158, 179, 151, 156, 184, 181, 183, 182, 180,
       188, 196, 197, 198, 194, 195, 199, 202, 203, 201, 206, 205, 200,
       204, 208, 210, 207, 209, 217, 224, 220, 215, 212, 226, 227, 244,
       245, 243, 246, 252, 251, 248, 253, 249, 250, 264, 266, 258, 256,
       267, 259, 262, 254, 263, 265, 272, 281, 271, 283, 269, 279, 268,
       277, 280, 282, 275, 273, 312, 307, 300, 318, 310, 301, 303, 323,
       338, 345, 331, 337, 342, 328, 336, 350, 352, 351, 365, 360, 364,
       358, 353, 355, 357, 356, 363, 361, 394, 400, 397, 383, 390, 382,
       391, 381, 388, 389, 387, 403, 384, 402, 407, 393, 399, 398, 405,
       408, 404, 386, 396, 392, 401, 409, 385, 411, 413, 412, 41

In [75]:
# for i in dfpatient['hospitalid'].unique():
#     dfpatient1 = dfpatient[dfpatient["hospitalid"] == i]
#     dfpatient1 = dfpatient1.dropna(subset=["admissionheight"])
#     dfpatient1 = dfpatient1.dropna(subset=["admissionweight"])
#         # Merge dfpatient and df_pivot_clean on 'patientunitstayid'
#     merged_df = dfpatient1.merge(df_pivotlab_clean, on='patientunitstayid', how='inner')

#     # Merge the result with df_APR_clean on 'patientunitstayid'
#     final_df = merged_df.merge(df_APR_clean, on='patientunitstayid', how='inner')

#     # Display the final merged dataframe
#     final_df['apacheadmissiondx']

#     # final_df
#     final_df

#     # nltk.download('punkt')
#     # nltk.download('averaged_perceptron_tagger')
#     # nltk.download('wordnet')
    
#     selected_column=['patientunitstayid', 'unitvisitnumber',  'gender', 'age',
#         'ethnicity',  'apacheadmissiondx',
#         'admissionheight', 
#             'unitadmitsource', 
#         'admissionweight', 'sodium', 'creatinine', 'BUN', 'potassium', 'Hct',
#         'glucose', 'chloride', 'WBC x 1000', 'Hgb', 'RBC', 'calcium',
#         'platelets x 1000', 'MCV', 'MCHC', 'bicarbonate', 'RDW', 'MCH',
#         'anion gap', '-lymphs', '-monos', 'predictedicumortality',
#         'predictediculos', 'actualiculos', 'predictedhospitalmortality',
#         'predictedhospitallos', 'actualhospitallos']
#     df=final_df[selected_column]
#     print(i,(df['unitvisitnumber'] == 1).sum(),(df['unitvisitnumber'] == 2).sum(),(df['unitvisitnumber'] == 3).sum())


59 307 6 15
60 197 5 11
73 3910 207 68
67 231 2 7
66 561 7 7
63 623 25 45
71 335 11 4
56 151 1 3
68 240 3 5
69 281 5 9
58 147 3 2
61 118 1 3
92 435 12 1
79 1043 61 7
108 293 6 0
83 0 0 0
95 222 2 0
84 0 0 0
93 0 0 0
86 0 0 0
90 0 0 0
85 0 0 0
102 24 0 0
96 1 0 0
94 5 0 0
91 7 0 0
110 0 0 0
122 0 0 0
123 0 0 0
125 0 0 0
135 0 0 0
112 0 0 0
133 0 0 0
120 0 0 0
131 0 0 0
138 0 0 0
115 0 0 0
136 0 0 0
144 575 2 0
143 546 9 1
142 415 2 0
140 495 2 0
141 1177 5 1
175 0 0 0
165 170 12 8
152 129 1 4
167 310 5 19
171 102 1 3
154 92 1 3
148 172 4 2
157 133 0 9
176 266 5 10
164 15 0 0
155 25 10 3
146 74 3 4
174 9 0 0
158 25 0 0
179 0 0 0
151 0 0 0
156 0 0 0
184 734 33 5
181 794 43 7
183 1291 95 29
182 252 5 9
180 354 12 2
188 1476 1046 77
196 269 3 0
197 687 22 8
198 253 9 2
194 288 5 1
195 505 32 5
199 683 45 12
202 121 5 2
203 113 0 0
201 120 2 0
206 379 7 5
205 0 0 0
200 230 5 2
204 89 2 0
208 0 0 0
210 0 0 0
207 0 0 0
209 0 0 0
217 278 6 1
224 22 0 0
220 179 4 0
215 146 4 1
212 0 0 0
226 664 

In [71]:
#选择医院代号188,不可在不同医院之间预测ICU次数

dfpatient1 = dfpatient[dfpatient["hospitalid"] == 71]
dfpatient1 = dfpatient1.dropna(subset=["admissionheight"])
dfpatient1 = dfpatient1.dropna(subset=["admissionweight"])
# dfpatient1

In [ ]:
#clean lab and APR table
df_pivot_lab = dflab.pivot_table(index="patientunitstayid", columns="labname", values="labresult", aggfunc="first").reset_index()

def clean_df(df, K=20):

    # 确保存在 'patientunitstayid' 列
    if 'patientunitstayid' not in df.columns:
        raise ValueError("DataFrame 必须包含 'patientunitstayid' 列")

    # 排除 'patientunitstayid'，计算其他列的 NA 数量
    columns_to_consider = df.columns.drop('patientunitstayid')
    if len(columns_to_consider) < K:
        raise ValueError(f"可用列数 ({len(columns_to_consider)}) 不足以选择 K={K} 列")

    # 计算各列的 NA 数量并排序
    na_counts = df[columns_to_consider].isna().sum()
    selected_columns = na_counts.nsmallest(K).index.tolist()

    # 构建新 DataFrame
    cleaned_df = df[['patientunitstayid'] + selected_columns]

    # 过滤选中的 K 列中所有 NA 的行
    cleaned_df = cleaned_df.dropna(subset=selected_columns)

    return cleaned_df
df_pivotlab_clean=clean_df(df_pivot_lab,K=20)




filtered_df = dfAPR[dfAPR["apacheversion"] == "IVa"]
lsit_columns=['predictedicumortality',
       'predictediculos', 'actualiculos', 'predictedhospitalmortality', 'predictedhospitallos', 'actualhospitallos']


df_APR_clean=filtered_df[['patientunitstayid'] + lsit_columns]
df_APR_clean

In [72]:
#得到最终的数据表格，但未完成category factor的转化
from collections import defaultdict, Counter
import nltk
from nltk.corpus import wordnet
import pandas as pd
nltk.download('averaged_perceptron_tagger_eng')
# Merge dfpatient and df_pivot_clean on 'patientunitstayid'
merged_df = dfpatient1.merge(df_pivotlab_clean, on='patientunitstayid', how='inner')

# Merge the result with df_APR_clean on 'patientunitstayid'
final_df = merged_df.merge(df_APR_clean, on='patientunitstayid', how='inner')

# Display the final merged dataframe
final_df['apacheadmissiondx']

# final_df
final_df

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
final_df.columns
selected_column=['patientunitstayid', 'unitvisitnumber',  'gender', 'age',
       'ethnicity',  'apacheadmissiondx',
       'admissionheight', 
        'unitadmitsource', 
       'admissionweight', 'sodium', 'creatinine', 'BUN', 'potassium', 'Hct',
       'glucose', 'chloride', 'WBC x 1000', 'Hgb', 'RBC', 'calcium',
       'platelets x 1000', 'MCV', 'MCHC', 'bicarbonate', 'RDW', 'MCH',
       'anion gap', '-lymphs', '-monos', 'predictedicumortality',
       'predictediculos', 'actualiculos', 'predictedhospitalmortality',
       'predictedhospitallos', 'actualhospitallos']

df=final_df[selected_column]
print((df['unitvisitnumber'] == 1).sum())
print((df['unitvisitnumber'] == 2).sum())
(df['unitvisitnumber'] == 3).sum()

335
11


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/xuranm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


np.int64(4)

In [ ]:

df['apacheadmissiondx']
num_unique = df['apacheadmissiondx'].nunique()
num_unique

word_list = []
for row in df['apacheadmissiondx']:
    if pd.notna(row):  # 忽略 NaN 值
        words = row.split()  # 按空格拆分单词（自动过滤空字符串）
        # 去重并保持原始顺序
        seen = set()
        unique_words = [word for word in words if not (word in seen or seen.add(word))]
        word_list.extend(unique_words)

# 词性标注
tagged_words = nltk.pos_tag(word_list)

# 过滤出名词和形容词
nouns_and_adjs = [
    word for word, tag in tagged_words
    if tag.startswith('NN') or tag.startswith('JJ')
]

# 统计频率
word_freq = Counter(nouns_and_adjs)
word_freq

K = 60

# 筛选频率大于 K 的单词
filtered_word_freq = {word: freq for word, freq in word_freq.items() if freq > K}

filtered_word_freq


def filter_same_freq_words(filtered_word_freq, df):
    # 按频率分组
    freq_to_words = defaultdict(list)
    for word, freq in filtered_word_freq.items():
        freq_to_words[freq].append(word)

    # 记录需要保留的单词
    words_to_keep = set()

    # 检查每组频率相同的单词是否总是出现在相同的行中
    for freq, words in freq_to_words.items():
        if len(words) >= 1:  # 只检查频率相同的多个单词
            # 创建单词到行索引的映射
            word_to_rows = defaultdict(set)
            for idx, row in df['apacheadmissiondx'].items():
                if pd.notna(row):
                    row_words = row.split()
                    for word in words:
                        if word in row_words:
                            word_to_rows[word].add(idx)
            # 检查所有单词的行索引是否相同
            first_word_rows = word_to_rows[words[0]]
            all_same = True
            for word in words:
                if word_to_rows[word] != first_word_rows:
                    all_same = False
                    break
            # 如果所有单词的行索引相同，则只保留第一个单词
            if all_same:
                words_to_keep.add(words[0])
            else:
                words_to_keep.update(words)
        else:  # 频率唯一的单词直接保留
            words_to_keep.add(words[0])

    # 构建结果字典，只保留需要保留的单词
    result = {word: freq for word, freq in filtered_word_freq.items() if word in words_to_keep}
    return result




dictionary=filter_same_freq_words(filtered_word_freq,df)
dictionary

In [ ]:
import re

def contains_word(a, b):
    # Match the word `a` even if surrounded by punctuation/whitespace
    pattern = rf'(?<!\w){re.escape(a)}(?!\w)'  # More robust check
    return bool(re.search(pattern, b))


df_new = df.copy()
word_list=list(dictionary.keys())
word_length = len(word_list)
vector_length = word_length + 1
apache_vectors = []
for row in df['apacheadmissiondx']:
    # print(row)
    vector = [0] * vector_length
    for word in word_list:
        if contains_word(word,row):
            vector[word_list.index(word)]=1
    apache_vectors.append(vector)

# 添加到新的 DataFrame
for i in range(vector_length):
    column_name = f"{'apacheadmissiondx'}_{i}"
    df_new[column_name] = [v[i] for v in apache_vectors]

# 删除原始的 apacheadmissiondx 列
df_new.drop('apacheadmissiondx', axis=1, inplace=True)


In [ ]:
import re

def contains_word(a, b):
    pattern = rf'\b{re.escape(a)}\b'
    return bool(re.search(pattern, b))


def one_hot_encode_categorical(df, category_column_name, word_list):
    df_new = df.copy()

    for column in category_column_name:
        if column == 'apacheadmissiondx':
            # 处理 apacheadmissiondx 列
            word_length = len(word_list)
            vector_length = word_length + 1

            # 生成对应的向量
            apache_vectors = []
            for row in df[column]:
                vector = [0] * vector_length
                
                apache_vectors.append(vector)

            # 添加到新的 DataFrame
            for i in range(vector_length):
                column_name = f"{column}_{i}"
                df_new[column_name] = [v[i] for v in apache_vectors]

            # 删除原始的 apacheadmissiondx 列
            df_new.drop(column, axis=1, inplace=True)
        else:
            # 处理普通列，转换为独热编码
            unique_values = df[column].unique()
            for value in unique_values:
                column_name = f"{column}_{value}"
                df_new[column_name] = (df[column] == value).astype(int)
            # 删除原始的列
            df_new.drop(column, axis=1, inplace=True)

    return df_new



['patientunitstayid', 'unitvisitnumber',  'gender', 'age',
       'ethnicity',  'apacheadmissiondx',
       'admissionheight', 
        'unitadmitsource', 
       'admissionweight', 'sodium', 'creatinine', 'BUN', 'potassium', 'Hct',
       'glucose', 'chloride', 'WBC x 1000', 'Hgb', 'RBC', 'calcium',
       'platelets x 1000', 'MCV', 'MCHC', 'bicarbonate', 'RDW', 'MCH',
       'anion gap', '-lymphs', '-monos', 'predictedicumortality',
       'predictediculos', 'actualiculos', 'predictedhospitalmortality',
       'predictedhospitallos', 'actualhospitallos']

dfnew=one_hot_encode_categorical(df,['gender', 'ethnicity','unitadmitsource'],dictionary)
dfnew.apacheadmissiondx.to_csv("1.csv")
dfnew.loc[dfnew["age"] =='> 89', "age"] = 89
dfnew.to_csv("output.csv", index=False, encoding="utf-8")

In [ ]:
dfnew.loc[dfnew["age"] =='> 89', "age"] = 89
dfnew.to_csv("output.csv", index=False, encoding="utf-8")

In [ ]:

# Define category mapping
category_map = {
    "ARDS": "Respiratory",
    "pneumonia": "Respiratory",
    "post-extubation edema": "Respiratory",
    "bacteremia": "Sepsis-related",
    "infection": "Sepsis-related",
    "hemoptysis": "Respiratory",
    "tracheostomy": "Respiratory",
    "Cardiac arrest": "Cardiac",
    "CHF": "Cardiac",
    "Infarction": "Cardiac",
    "hypertension": "Cardiac",
    "stroke": "Neurological",
    "Sepsis": "Sepsis-related",
    "Septic shock": "Sepsis-related",
    "CVA": "Neurological",
    "coma": "Neurological",
    "intracranial hemorrhage": "Neurological",
    "seizure": "Neurological",
    "subarachnoid hemorrhage": "Neurological",
    "GI bleeding": "Gastrointestinal",
    "peritonitis": "Gastrointestinal",
    "variceal bleeding": "Gastrointestinal",
    "surgery": "Gastrointestinal",
    "Acid-base": "Metabolic/Electrolyte",
    "electrolyte": "Metabolic/Electrolyte",
    "diabetes": "Metabolic/Electrolyte",
    "cocaine": "Toxicology",
    "benzodiazepines": "Toxicology",
    "calcium channel blockers": "Toxicology",
    "analgesic": "Toxicology",
    "trauma": "Surgical/Trauma",
    "transphenoidal surgery": "Surgical/Trauma"
}

# Function to categorize each diagnosis
def categorize_diagnosis(diagnosis_str):
    diagnoses = diagnosis_str.split(", ")  # Split by comma
    category_set = set()  # Use a set to avoid duplicates
    
    for diagnosis in diagnoses:
        for key, category in category_map.items():
            if key.lower() in diagnosis.lower():  # Case-insensitive matching
                category_set.add(category)
    
    return ", ".join(category_set) if category_set else "Other"  # Combine categories

# Apply function to create the new categorized column
dfnew["apacheadmissiondx_category"] = dfnew["apacheadmissiondx"].apply(categorize_diagnosis)

# Save to a new CSV
dfnew.drop('apacheadmissiondx', axis=1, inplace=True)


In [ ]:
df_category_vectors = dfnew["apacheadmissiondx_category"].str.get_dummies(sep=", ")
df_combined = pd.concat([dfnew, df_category_vectors], axis=1)

# Drop the original apacheadmissiondx_category column
df_combined = df_combined.drop("apacheadmissiondx_category", axis=1)

# Save the final transformed DataFrame
df_combined.to_csv("final_transformed_data_458.csv", index=False)


In [ ]:
df_combined

In [ ]:
sum(df_combined.unitvisitnumber==3)